In [ ]:
import openai
import pandas as pd

In [ ]:
openai.organization = "" ## organization id
openai.api_key = ""  ## open api key
# Read Answers from CSV file
df = pd.read_csv(r'', header=None)
answers = df.iloc[1:, 1].tolist()

In [ ]:
Explanation_scheme_step_and_score_C1="""
As an essay grader, your task is to evaluate and assign a grade between 1 and 5 to the provided Korean text. Re-evaluate your thought process step by step to show how it reflects the grading scheme. Provide your grade as a single integer value.
The text you are grading is an answer to a question. Your evaluation should be based on the combined criteria of problem comprehension and explanation quality.


Please consider the following aspects while grading:
- Problem Comprehension: Assess how effectively the text demonstrates understanding of the problem and its relevance. Focus on the introduction and information presented throughout the text.
- Explanation Quality: Evaluate the informativeness of the explanation provided. Ensure that the text offers sufficient information to clarify the problem concisely.


Please use the following grading scheme:
5 points: The text demonstrates a very effective understanding of the problem, is closely related to the issue, and provides necessary information.
4 points: The text demonstrates a generally effective understanding of the problem, is closely related to the issue, and includes mostly necessary information.
3 points: The text shows an understanding of the problem related to the issue but lacks some information.
2 points: The text's understanding of the problem contains information that is not closely related to the issue.
1 point: The text provides no explanation of the problem or displays a very weak understanding.

Now, carefully review the following question and answer, and assign a grade accordingly:
"""

Explanation_scheme_step_and_score_C2="""
As an essay grader, your task is to evaluate and assign a grade between 1 and 5 to the provided Korean text. Re-evaluate your thought process step by step to show how it reflects the grading scheme. Provide your grade as a single integer value.
The text you are grading is an answer to a question. Your evaluation should be based on the appropriateness of the argument and the clarity of the argument. 

Please consider the following aspects when assigning the grade:
- Appropriateness of the argument: Assess how in line the claim is with the issue. 
- Clarity of the argument: Evaluate if the claim is consistent and distinct throughout the text.

Please use the following grading scheme:
5 points: The claim is in line with the issue and remains highly consistent and clear throughout the text.
4 points: The claim is in line with the issue and generally maintains consistency and clarity throughout the text.
3 points: The claim is generally in line with the issue, but there are some inconsistencies or lack of clarity.
2 points: The claim somewhat deviates from the issue; or while related to the issue, it lacks significant consistency or clarity.
1 point: The claim does not align with the issue, or while it might be related, it severely lacks consistency and clarity.

Now, carefully review the following question and answer, and assign a grade accordingly:
"""

Explanation_scheme_step_and_score_C3="""
As an essay grader, your task is to evaluate and assign a grade between 1 and 5 to the provided Korean text. Re-evaluate your thought process step by step to show how it reflects the grading scheme. Provide your grade as a single integer value.
The text you are grading is an answer to a question. Your evaluation should be based on the appropriateness of reasoning and evidence. 

Please consider the following aspects when assigning the grade:
- Coherence of Claim-Reasoning/Evidence Relationship: Assess whether there is a suitable connection established between the claim and the accompanying reasoning/evidence. Verify if the provided reasons or evidence adequately support the claim or subclaims and if the claim aligns with the relevant issue.

Please use the following grading scheme:
5 points: Very appropriate reasoning and evidence, highly persuasive.
4 points: Reasoning and evidence are generally appropriate, somewhat persuasive.
3 points: Some of the reasoning and evidence are not appropriate, but generally acceptable.
2 points: Many cases where reasoning and evidence are not appropriate, low persuasiveness.
1 point: Most of the reasoning and evidence are not appropriate, very low persuasiveness.

Now, carefully review the following question and answer, and assign a grade accordingly:
"""

Explanation_scheme_step_and_score_C4="""
As an essay grader, your task is to evaluate and assign a grade between 1 and 5 to the provided Korean text. Re-evaluate your thought process step by step to show how it reflects the grading scheme. Provide your grade as a single integer value.
The text you are grading is an answer to a question. Your evaluation should be based on the combined criteria of sufficiency of reasons/evidence and depth of reasons/evidence.

Please consider the following aspects when assigning the grade:
- Sufficiency of reasons / evidence: Assess if there are sufficient and appropriate reasoning or evidence supporting the main claim. 
- Clarity of the argument: Evaluate if the ideas behind the reasons or evidence are elaborated in-depth.

Please use the following grading scheme:
5 points: Two or more appropriate reasons for the main claim are presented, and each reason is deeply supported with relevant evidence.
4 points: Two or more appropriate reasons for the main claim are presented; however, there is a lack of in-depth support with relevant evidence for the reasons.
3 points: One appropriate reason for the main claim is presented, and it is relatively well-supported with relevant evidence in a somewhat detailed manner.
2 points: One appropriate reason for the main claim is presented, but there is a lack of in-depth support with relevant evidence for the reasons.
1 point: None of the reasons for the main claim are appropriate.

Now, carefully review the following question and answer, and assign a grade accordingly:
"""




Explanation_scheme_step_and_score_C5="""
As an essay grader, your task is to evaluate and assign a grade between 1 and 5 to the provided Korean text. Re-evaluate your thought process step by step to show how it reflects the grading scheme. Provide your grade as a single integer value.
The text you are grading is an answer to a question. Your evaluation should be based on the combined criteria of diversity of approaches and the level of consideration for different viewpoints.

Please consider the following aspects when assigning the grade:
- Diversity of approaches: Assess whether different approaches have been considered. 
- Level of consideration for other viewpoints: Evaluate if other perspectives are elaborated in depth. If specific information supporting different perspectives is included, grade 3 or more will be assigned. If the existence of different perspectives is mentioned without specific information (e.g., "there might be people who think differently"), grade 2 will be assigned.

Please use the following grading scheme:
5 points: Different perspectives are considered, and their content is elaborated in-depth.
4 points: Different perspectives are considered, and their content is generally elaborated in-depth.
3 points: Different perspectives are considered, and their content is adequately addressed without in-depth elaboration.
2 points: The awareness of the possibility of different perspectives is indirectly evident. (e.g., by toning down the strength of the claim or using non-dogmatic expressions)
1 point: Only a singular viewpoint is mentioned in a dogmatic manner.

Now, carefully review the following question and answer, and assign a grade accordingly:
"""

Explanation_scheme_step_and_score_O1="""
As an essay grader, your task is to evaluate and assign a grade between 1 and 5 to the provided Korean text. Re-evaluate your thought process step by step to show how it reflects the grading scheme. Provide your grade as a single integer value.
The text you are grading is an answer to a question. Your evaluation should be based on the combined criteria of appropriateness of paragraph structure and coherence of paragraph organisation.

Please consider the following aspects when assigning the grade:
- Appropriateness of paragraph structure: Assess whether the text has a structure of Introduction - Body - Conclusion. 
- Coherence of Paragraph Organisation: Evaluate if the arrangement of paragraphs are smooth and cohesive. If the overall flow of the writing is generally smooth and cohesive, but the paragraphs are not structured appropriately, grade 3 will be assigned.

Please use the following grading scheme:
5 points: The structure of Introduction-Body-Conclusion is very appropriately distinguished, and the arrangement of paragraphs is very smooth and cohesive.
4 points: The structure of Introduction-Body-Conclusion is generally appropriately distinguished, and the arrangement of paragraphs is mostly smooth and cohesive.
3 points: The structure and arrangement are somewhat disjointed and incoherent, but they do not significantly hinder the readability of the text.
2 points: The overall organisation of the text is somewhat disjointed and incoherent, resulting in a low level of completeness, and there are many incoherent parts in the arrangement of paragraphs.
1 point: The content paragraphs are hardly distinguishable, leading to a lack of structure in the text.

Now, carefully review the following question and answer, and assign a grade accordingly:
""" 

Explanation_scheme_step_and_score_O2="""
As an essay grader, your task is to evaluate and assign a grade between 1 and 5 to the provided Korean text. Re-evaluate your thought process step by step to show how it reflects the grading scheme. Provide your grade as a single integer value.
The text you are grading is an answer to a question. Your evaluation should be based on the combined criteria of completeness, unity and coherence of the paragraphs.

Please consider the following aspects when assigning the grade:
- Completeness: Assess whether the topic sentence of each paragraph is supported with supporting sentences. 
- Unity: Evaluate if there is a main theme or a main idea developed in each paragraph. 
- Coherence: Check if the sentences within the paragraph connect with each other smoothly. (e.g., through comparison or contrast)

Please use the following grading scheme:
5 points: All paragraphs exhibit completeness, unity and coherence.
4 points: For the most part, paragraphs show completeness, unity and coherence.
3 points: Some paragraphs lack completeness, unity and coherence, but they do not significantly hinder the readability of the text.
2 points: The overall completeness, unity and coherence are low, resulting in a low readability of the text.
1 point: Most paragraphs fail to show completeness, unity and coherence.

Now, carefully review the following question and answer, and assign a grade accordingly:
""" 

Explanation_scheme_step_and_score_L1="""
As an essay grader, your task is to evaluate and assign a grade between 1 and 5 to the provided Korean text. Re-evaluate your thought process step by step to show how it reflects the grading scheme. Provide your grade as a single integer value.
The text you are grading is an answer to a question. Your evaluation should be based on the combined criteria of fluency of sentences, effectiveness of sentences and appropriate usage of vocabulary.

Please consider the following aspects when assigning the grade:
- Fluency of sentences: Assess whether the sentence lengths are appropriate and if the expressions used are smooth. (e.g., avoiding awkward phrasing) 
- Effectiveness of sentences: Evaluate if the meaning of the sentences is clear and the expressions used are effective. 
- Appropriate usage of vocabulary: Check if the vocabulary usage is appropriate for the context.

Please use the following grading scheme:
5 points: The sentences are fluent and effective, and the vocabulary usage is appropriate.
4 points: The sentences are generally fluent and effective, and the vocabulary usage is mostly appropriate.
3 points: Some sentences are not fluent or effective, and there are occasional inappropriate vocabulary usages, but they do not significantly hinder the readability of the text.
2 points: Many sentences are not fluent or effective, and there are frequent inappropriate vocabulary usages.
1 point:  Most sentences are poorly written and it is challenging to follow their argument or understand the meaning of the text. There is a major missuse of vocabulary.

Now, carefully review the following question and answer, and assign a grade accordingly:
""" 

Explanation_scheme_step_and_score_L2="""
As an essay grader, your task is to evaluate and assign a grade between 1 and 5 to the provided Korean text. Re-evaluate your thought process step by step to show how it reflects the grading scheme. Provide your grade as a single integer value.
The text you are grading is an answer to a question. Your evaluation should be based on the combined criteria of adherence to linguistic norms and adherence to writing conventions.

Please consider the following aspects when assigning the grade:
- Adherence to linguistic norms: Check if spelling, spacing and other language conventions have been followed correctly.
- Adherence to writing conventions: Check if the language style is in line with the genre of the writing.

Please use the following grading scheme:
5 points: The text adheres very accurately to linguistic norms and writing conventions.
4 points: The text mostly adheres to linguistic norms and writing conventions.
3 points: There are some errors with linguistic norms and writing conventions, but no major errors were found.
2 points: There are many errors with linguistic norms and writing conventions, which lowers the readability of the text.
1 point:  The errors with linguistic norms and writing conventions are serious, making it difficult to understand the text.

Now, carefully review the following question and answer, and assign a grade accordingly:
""" 

Explanation_scheme_step_and_score_C5_revised="""
As an essay grader, your task is to evaluate and assign a grade between 1 and 5 to the provided Korean text. Re-evaluate your thought process step by step to show how it reflects the grading scheme. Provide your grade as a single integer value.
The text you are grading is an answer to a question. Your evaluation should be based on the combined criteria of diversity of opinions and the level of consideration for different viewpoints.

Please consider the following aspects when assigning the grade:
- Diversity of opinions: Assess whether opposing viewpoints or contrasting perspectives opposed to the text's main perspective have been considered. 
- Level of consideration for other viewpoints: Evaluate if the opposing perspectives are elaborated in depth. If specific information supporting opposing perspectives is included, grade 3 or more will be assigned. If the existence of opposing perspectives is mentioned without specific information (e.g., "there might be people who think differently"), grade 2 will be assigned.

Please use the following grading scheme:
5 points: Opposing perspectives are considered, and their content is elaborated in-depth.
4 points: Opposing perspectives are considered, and their content is generally elaborated in-depth.
3 points: Opposing perspectives are considered, and their content is adequately addressed without in-depth elaboration.
2 points: The awareness of the possibility of opposing perspectives is indirectly evident. (e.g., by toning down the strength of the claim or using non-dogmatic expressions)
1 point: Only a singular viewpoint is mentioned in a dogmatic manner.

Now, carefully review the following question and answer, and assign a grade accordingly:
"""

In [ ]:
from tenacity import (
    retry,
    stop_after_attempt,
    wait_random_exponential,
    retry_if_exception_type
)  # for exponential backoff

@retry(
    retry=retry_if_exception_type((openai.error.APIError, openai.error.APIConnectionError, openai.error.RateLimitError, openai.error.ServiceUnavailableError, openai.error.Timeout)), 
    wait=wait_random_exponential(multiplier=1, max=60), 
    stop=stop_after_attempt(10)
)
def get_grade_gpt4(prompt_type, QA):
    response = openai.ChatCompletion.create(
        model="gpt-4",
        messages=[
            {"role": "system", "content": prompt_type},
            {"role": "user", "content": QA}
        ],
        temperature = 0.1
    )
    response_text = response['choices'][0]['message']['content'].strip()
    return response_text

def get_grade_gpt3(prompt_type, QA):
    response = openai.ChatCompletion.create(
        model="gpt-3.5-turbo",
        messages=[
            {"role": "system", "content": prompt_type},
            {"role": "user", "content": QA}
        ],
        temperature = 0.1 
        
    )
    response_text = response['choices'][0]['message']['content'].strip()
    return response_text




In [ ]:
# GPT4
# Process prompts and get responses
C2_responses_gpt4= []
for answer in answers:
    response_text = get_grade_gpt3(Explanation_scheme_step_and_score_C2, answer)
    C2_responses_gpt4.append(response_text)


# Create a new DataFrame with the responses
C2_responses_gpt4_df = pd.DataFrame({'C2_responses_gpt4_gpt3.5': C2_responses_gpt4})

# Load existing Excel file
file_path = r''
writer = pd.ExcelWriter(file_path, engine='openpyxl')

# Write the new DataFrame to the Excel file
C2_responses_gpt4_df.to_excel(writer, index=False, header=['C2_responses_gpt4'])
C2_responses_gpt4_df.to_csv(r"")
# Save and close the Excel file
writer.close()